# Part C1: Feature Engineering on the clean dataset

This part aims to bring massive time-series data into reduced data by defining several statistical features representing the daily profiles, besides enriching the dataset by adding new dynamic influencing factors, like weather, day type, holidays that could integrate new knowledge. Therefore:

1. **Data Segmentation**
2. **Enriching the dataset**

# 1. Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import stats
import scipy.stats as st
import seaborn as sns
from calendar import day_abbr, month_abbr, mdays
import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import math
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
import plotly.graph_objs as go
from sklearn.metrics import silhouette_score
import holidays
from calendar import day_abbr, month_abbr, mdays

# 2. Functions

In [3]:
def hourly_trend(df,attribute):
    hour_consumption =df.groupby(['hour']).mean()
    q25 = df.groupby(['hour']).quantile(0.25)
    q75 = df.groupby(['hour']).quantile(0.75)

    f, ax = plt.subplots(figsize=(10,7)) 
    hour_consumption.plot(ax=ax, lw=2, color='b', legend=False)
    ax.fill_between(hour_consumption.index, q25.values.ravel(), q75.values.ravel(), color='b', alpha=0.3)
    ax.grid(ls=':')
    ax.set_xlabel('Hour', fontsize=15)
    ax.set_ylabel( attribute, fontsize=15);
    [l.set_fontsize(13) for l in ax.xaxis.get_ticklabels()]
    [l.set_fontsize(13) for l in ax.yaxis.get_ticklabels()]

    ax.set_title('Hourly ' + attribute + ' consumption of HH', fontsize=15)
    plt.show()

# 3. Import Clean Data

In [4]:
df = pd.read_csv('../CleanDataset.csv', index_col = 0, parse_dates=True)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../CleanDataset.csv'

# 4. Data Segmentation

Time-series energy data often have high dimensions that can bring challenges to clustering algorithms based on the distance function (i.e., Euclidean distance), causing issues such as producing poor clustering results and increasing computational costs. Feature definition was therefore used for dimensionality reduction in this study.

## 4.1. Add time-scaled features

In [ ]:
df['dayofweek'] = df.index.dayofweek
df['hour'] = df.index.hour
df['month'] = df.index.month
df['time'] = df.index.time
df.head()

## 4.2. HH Electricity data

Based on the hourly trend of Electricity data (same for HH and DE blocks), we divide the day into 5 segmentations, where for each segmentation we compute the <ins>mean</ins> value:
1. 00:00 - 05:00 -> off-time
2. 05:00 - 10:00 -> rise-time
3. 10:00 - 15:00 -> day-time
4. 15:00 - 22:00 -> evening
5. 22:00 - 00:00 -> off-time

In [ ]:
df_HH_elec = df[['HH Electricity (kWh)', 'hour']]
attribute = 'Electricity (kWh)'
hourly_trend(df_HH_elec,attribute)

Besides the 5 new dimensions, we introduce a 6th feature, the daily peak-to-valley difference rate which is defined as the ratio of the differencce between the daily maximum and the minimum load to the daily maximum load.

In [ ]:
df_HH_elec = df[['HH Electricity (kWh)', 'dayofweek','hour','time']]
df_HH_elec

In [ ]:
df_HH_elec['Date'] = df.index.date
#df_HH_elec['Time'] = pd.to_datetime(df.index,format='%H:%M:%S').time

In [ ]:
df_HH_elec

In [ ]:
#df_HH_elec_new = df_HH_elec.groupby('Date').resample("5H").mean()['HH Electricity (kWh)']
df_HH_elec_new = df_HH_elec.set_index(['Date',df_HH_elec.index])['HH Electricity (kWh)']
df_HH_elec_new

In [ ]:
def stats_func(s, h1, h2, h3, h4):
    mean_list = []
    # time segments = 22:00-05:00, 05:00-10:00, 10:00-15:00,15:00-22:00
    sum_0 = 0
    sum_1 = 0
    sum_2 = 0
    sum_3 = 0
    min_ = 10000
    max_ = 0
    max_0 = 0
    max_1 = 0
    max_2 = 0
    max_3 = 0
    for i in range(0,s.shape[0]):

        hh = s.index.get_level_values(1)[i].time().hour
        if hh < h1:
            sum_0 += s[i]
            #print("1",s[i])
            if s[i] > max_0:
                max_0 = s[i]
        elif h1 <= hh < h2:
            sum_1 += s[i] 
            #print("2",s[i])
            if s[i] > max_1:
                max_1 = s[i]
        elif h2<= hh < h3:
            sum_2 += s[i]
            #print("3",s[i])
            if s[i] > max_2:
                max_2 = s[i]
        elif h3 <= hh < h4:
            sum_3 += s[i]
            #print("4",s[i])
            if s[i] > max_3:
                max_3 = s[i]
        else: 
            sum_0 += s[i]
            #print("5",s[i])
            if s[i] > max_0:
                max_0 = s[i]
        
        #min
        if s[i] < min_:
            min_ = s[i]
        #max
        if s[i] > max_:
            max_ = s[i]
            
    ## compute means for each segment    
    mean_0 = round(sum_0/(2*(h1 + 24 - h4)),2)
    mean_1 = round(sum_1/(2*(h2-h1)),2)
    mean_2 = round(sum_2/(2*(h3-h2)),2)
    mean_3 = round(sum_3/(2*(h4-h3)),2)
    
    ## compute paek-to-valley value = (max-min)/max
    ptv = round((max_-abs(min_))/max_,2)
    
    stats_list = pd.Series([mean_0, mean_1, mean_2, mean_3, min_, max_, ptv])    
    return stats_list

In [ ]:
# define time segmentations
h1=5
h2=10
h3=15
h4=22
df_HH_elec_new2 = df_HH_elec_new.groupby('Date').apply(lambda x: stats_func(x, h1, h2, h3, h4))
df_HH_elec_new2

In [ ]:
#unstack multindex series to dataframe
df_HH_elec_new2 = df_HH_elec_new2.unstack(level=1)
df_HH_elec_new2

In [ ]:
col = ['Elec_mean_0', 'Elec_mean_1', 'Elec_mean_2', 'Elec_mean_3',  'Elec_min_', 'Elec_max_', 'Elec_ptv']
df_HH_elec = pd.DataFrame(df_HH_elec_new2)
df_HH_elec.columns = col
df_HH_elec

In [ ]:
# df_HH_elec_new = df_HH_elec.groupby('Date').resample("5H").mean()['HH Electricity (kWh)']
# df_HH_elec_new = df_HH_elec_new.reset_index().rename(columns={'HH Electricity (kWh)':'Elec_mean'})
# df_HH_elec_new = df_HH_elec_new.rename(columns={'From Timestamp':'Time'})
# df_HH_elec_new['Time'] = df_HH_elec_new['Time'].dt.time
#df_HH_elec_new = df_HH_elec_new.groupby(['Date','Time']).mean()
# df_HH_elec_new = df_HH_elec_new.set_index(['Date','Time'])
#df_HH_elec_new = df_HH_elec_new.unstack()
#df_HH_elec_new1.columns = [col if type(col) is datetime else col for col in df_HH_elec_new.columns.values]
# df_HH_elec_new = df_HH_elec_new.unstack().set_axis(['Elec_mean_00', 'Elec_mean_05', 'Elec_mean_10', 'Elec_mean_15', 'Elec_mean_20'], axis=1)
# df_HH_elec_new

**Therefore we <ins>reduced a 48-dimension (data for every half an hour for each date) dataset to a 5-dimension dataset</ins>.**

**Normalize data so  that all features belong to the same scale and Euclidean distance will not be biased in clustering methods:**

In [ ]:
# Standardize data
scaler = MinMaxScaler() #StandardScaler()
df_HH_elec_scaled = scaler.fit_transform(df_HH_elec.values)
df_HH_elec_scaled = pd.DataFrame(df_HH_elec_scaled, index=df_HH_elec.index, columns=df_HH_elec.columns)
df_HH_elec_scaled

In [ ]:
# index = True: write row names (indexes)
df_HH_elec_scaled.to_csv('../HH_elec_Dataset.csv', index=True, header=True)

## 4.2. HH Electricity data

In [ ]:
df_DE_elec = df[['DE Electricity (kWh)', 'hour']]
attribute = 'Electricity (kWh)'
hourly_trend(df_DE_elec,attribute)

In [ ]:
df_DE_elec = df[['DE Electricity (kWh)', 'dayofweek','hour','time']]
df_DE_elec['Date'] = df.index.date
df_DE_elec = df_DE_elec.set_index(['Date',df_DE_elec.index])['DE Electricity (kWh)']
df_DE_elec = df_DE_elec.dropna(inplace=False)
df_DE_elec

In [ ]:
# define time segmentations
h1=5
h2=10
h3=15
h4=22
df_DE_elec = df_DE_elec.groupby('Date').apply(lambda x: stats_func(x, h1, h2, h3, h4))
#unstack multindex series to dataframe
df_DE_elec = df_DE_elec.unstack(level=1)
df_DE_elec.head()

In [ ]:
col = ['Elec_mean_0', 'Elec_mean_1', 'Elec_mean_2', 'Elec_mean_3',  'Elec_min_', 'Elec_max_', 'Elec_ptv']
df_DE_elec = pd.DataFrame(df_DE_elec)
df_DE_elec.columns = col

# Standardize data
scaler = MinMaxScaler() #StandardScaler()
df_DE_elec_scaled = scaler.fit_transform(df_DE_elec.values)
df_DE_elec_scaled = pd.DataFrame(df_DE_elec_scaled, index=df_DE_elec.index, columns=df_DE_elec.columns)
df_DE_elec_scaled.head()

In [ ]:
# index = True: write row names (indexes)
df_DE_elec_scaled.to_csv('../DE_elec_Dataset.csv', index=True, header=True)

# 5. External data

We enrich our dataset by adding external dynamic influencing factors for the energy consumption:

- Weekday
- Holiday
- Month
- Season
- Mean_temp
- Mean_hum

In [ ]:
df_HH_elec_scaled.head()

## 5.1. Weekday, Month, Season

In [ ]:
def season_of_date(date):
    month = date.month
    if 3<=month<=5:
        return 'Spring'
    elif 6<=month<=8:
        return 'Summer'
    elif 9<=month<=11:
        return 'Autumn'
    else:
        return 'Winter'

In [ ]:
df_HH_elec_scaled['Weekday'] = df_HH_elec_scaled.index.map(lambda row: day_abbr[row.dayofweek])
df_HH_elec_scaled['Month'] = df_HH_elec_scaled.index.map(lambda row: month_abbr[row.month])

# season
df_HH_elec_scaled['Season'] = df_HH_elec_scaled.index.map(lambda row: season_of_date(row))
df_HH_elec_scaled.head()

## 5.2. The Holidays package

Knowing when holidays and special events take place is often crucial when modelling time-series data. Here we make use of the `holidays` [package](https://github.com/dr-prodigy/python-holidays).

In [ ]:
holidays_df = pd.DataFrame([], columns = ['Date','Holiday'])
ldates = []
lnames = []
for date, name in sorted(holidays.England(years=np.arange(2018, 2019 + 1)).items()):
    ldates.append(date)
    lnames.append(name)
    
ldates = np.array(ldates)
lnames = np.array(lnames)
holidays_df.loc[:,'Date'] = ldates
holidays_df.loc[:,'Holiday'] = lnames
holidays_df.Holiday.unique()

In [ ]:
holidays_df = holidays_df.set_index(['Date'])
holidays_df

In [ ]:
df_enriched = pd.concat([df_HH_elec_scaled,holidays_df],axis=1)

# Impute null value with new category ("False")
df_enriched['Holiday'] = np.where(df_enriched['Holiday'].isnull(),"False",df_enriched['Holiday'])

In [ ]:
df_enriched.head()

## 5.3. Weather data

Download averaged temp and hum for each date from [rp5.ru](https://rp5.ru/Weather_archive_in_Southampton_(airport),_METAR), acquired by the weather station located in Southampton (where Hursley House is located) from January 2018 to December 2019.

In [ ]:
xls = pd.ExcelFile('../Weather data-Southampton.xls')
weather_df = pd.read_excel(xls, usecols = [0,1,4], parse_dates=True)
weather_df = weather_df.rename(columns={weather_df.columns[0]: 'Timestamp',
                                       weather_df.columns[1]: 'Temperature (C)',
                                       weather_df.columns[2]: 'Humidity (%)'})
weather_df=weather_df.set_index(['Timestamp'])
weather_df.index = pd.to_datetime(weather_df.index)
weather_df.head()

In [ ]:
weather_df['Date'] = weather_df.index.date

In [ ]:
weather_df = weather_df.set_index(['Date',weather_df.index])
weather_df

In [ ]:
#weather_df[weather_df.index.get_level_values('Date').date == datetime.date(2018, 1, 3)]

In [ ]:
weather_df = weather_df.groupby('Date').mean()
weather_df

In [ ]:
df_enriched = pd.concat([df_enriched, weather_df],axis=1)
df_enriched.head()

**NaN values:**

(df_enriched had 730 rows, while df_weather had 728)

In [ ]:
df_enriched.isnull().sum()

In [ ]:
df_enriched.index[df_enriched['Temperature (C)'].isnull()]

**Linear Interpolation:**

In [ ]:
#weather_df[weather_df.index.get_level_values('Date').date == datetime.date(2019, 12, 26)]
df_enriched = df_enriched.interpolate()

In [ ]:
df_enriched[df_enriched.index.date == datetime.date(2019, 12, 25)]

In [ ]:
df_enriched.isnull().sum()

In [ ]:
df_enriched

In [ ]:
# index = True: write row names (indexes)
df_enriched.to_csv('../EnrichedDataset.csv', index=True, header=True)

In [ ]:
# index_col = 0: use 1st column as row labels
# header = Row number(s) to use as the column names, and the start of the data.
df_enriched = pd.read_csv('../EnrichedDataset.csv', index_col = 0) # header=[0,1]
df_enriched

# Part C2: Prepare daily time-series

In [ ]:
df = pd.read_csv('../ImputedDataset.csv', index_col = 0, parse_dates=True)
df = df.drop(['dayofweek', 'hour','month'], axis=1)
df1 = df.copy()

# Scale data
#scaler = MinMaxScaler() #better for using later sigmoind in NN
#df1_scaled = scaler.fit_transform(df1.values)
#df1 = pd.DataFrame(df1_scaled, index=df1.index, columns=df1.columns)
df1.head()

In [ ]:
#df1['hour'] = df1.index.hour
df1['time'] = df1.index.time
df1['Date'] = df1.index.date
#df1 = df1.drop(df1.columns[6], axis=1) #drop hour column
df1 = df1.reset_index(drop=True)
df1.head()

In [ ]:
df1 = df1.set_index(['Date','time'])
df1.head()

In [ ]:
df1 = df1.unstack(level=1)
df1.head()

**Add month, dayofweek attributes:**

In [ ]:
df1['dayofweek'] = df1.index.dayofweek
df1['month'] = df1.index.month
df1.head()

**Create 2 different datasets one for each building:**

In [ ]:
df_HH = df1.drop(['DE Electricity (kWh)', 'DE Boiler (kWh)'], axis=1)
df_HH.head()

In [ ]:
df_DE = df1.drop(['HH Electricity (kWh)', 'HH Cooling (kWh)', 'HH Boiler (kWh)'], axis=1)
df_DE.head()

**Convert categorical variables to one-hot-encoding vectors:**

In [ ]:
dayofweek_dummies = pd.get_dummies(df1.dayofweek, prefix='Day')
month_dummies = pd.get_dummies(df1.month, prefix='Month')
df_HH = df_HH.drop(['month','dayofweek'], axis=1) #drop month, dayofweek columns
df_HH = pd.concat([df_HH, dayofweek_dummies,month_dummies], axis=1)

df_DE = df_DE.drop(['month','dayofweek'], axis=1) #drop month, dayofweek columns
df_DE = pd.concat([df_DE, dayofweek_dummies,month_dummies], axis=1)


df_HH.head()

**Save datasets locally:**

In [ ]:
# index = True: write row names (indexes)
df_HH.to_csv('../HH_Dataset.csv', index=True, header=True)
df_DE.to_csv('../DE_Dataset.csv', index=True, header=True)

In [ ]:
# index_col = 0: use 1st column as row labels
# header = Row number(s) to use as the column names, and the start of the data.
df_imputed = pd.read_csv('../HH_Dataset.csv', index_col = 0) # header=[0,1]
df_imputed

In [ ]:
df_imputed.min()